In [1]:
using Plots; qwt()
default(size=(500,300), leg=false)

# creates x/y vectors which can define a grid in a zig-zag pattern
function gridxy(lim, n::Int)
    xs = linspace(lim..., n)
    xypairs = vec([(x,y) for x in vcat(xs,reverse(xs)), y in xs])
    Plots.unzip(xypairs)
end

# plot a grid from x/y vectors
function plotgrid(x, y)
    plot([x y], [y x], c=:black)
end

plotgrid (generic function with 1 method)

# The problem... can we classify the functions?

In [2]:
# these are the functions we want to classify
f1(x) = 0.6sin(10x) + 0.1
f2(x) = f1(x) - 0.2

f2 (generic function with 1 method)

# Build a neural net

In [941]:
using OnlineAI

# first create a neural net to separate the functions
numInputs = 2
numOutputs = 1
hiddenLayerStructure = [3,3,2]
net = buildClassificationNet(numInputs, numOutputs, hiddenLayerStructure; hiddenActivation = TanhActivation())

# show the network
viz = visualize(net);

# On to the fun...

In [942]:
# pick the plotting limits
lim = (-1,1)
default(xlim = lim, ylim = lim)

# show the grid
n = 40
gridx, gridy = gridxy(lim, n)
p = plotgrid(gridx, gridy)

# show the funcs
funcs = [f1, f2]
plot!(funcs, lim..., w=3)

# kick off an animation... we can save frames whenever we want, lets save the start
anim = Animation()
frame(anim)

# open a gui window
gui()

# Yikes... that looks tricky to separate...

In [945]:
# function to sample from x's
xsample() = rand(Distributions.Uniform(lim...)) 

# pick one of the functions at random, sample from the x line, then update the
# neural net with [x, f(x)] as the inputs
function sampleAndUpdate()
    f = sample(funcs)
    x = xsample()
    y = float(f == f1)
    update!(net, Float64[x, f(x)], [y])
end

# take x matrix and convert to the first layer's activation
function activateHidden(net, x)
    input = x
    for layer in net.layers[1:end-1]
        proj = Array(nrows(x), layer.nout)
        for i in 1:nrows(x)
            OnlineAI.forward!(layer, row(proj,i), false)
            row!(proj, i, layer.a)
        end
        input = proj
    end
    vec(proj[:,1]), vec(proj[:,2])
end    

activateHidden (generic function with 1 method)

In [946]:
# update net with new samples
for i in 1:10000
    sampleAndUpdate()
end

# update the plot... project each series to the first hidden layer and reset the data
x = linspace(lim..., 100)
p[1] = activateHidden(net, hcat(gridx, gridy))
p[2] = activateHidden(net, hcat(gridy, gridx))
p[3] = activateHidden(net, hcat(x, map(f1,x)))
p[4] = activateHidden(net, hcat(x, map(f2,x)))

# show/update the plot
gui(p)
frame(anim);

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Array{T,N}}, ::Int64, ::Int64)
This may have arisen from a call to the constructor Array{T,N}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  convert{T,N}(::Type{Array{T,N}}, !Matched::DataArrays.DataArray{T,N}, ::Any)
  convert{T,R,N}(::Type{Array{T,N}}, !Matched::DataArrays.PooledDataArray{T,R,N}, ::Any)
  Array{T}(!Matched::Type{T}, ::Integer)
  ...
while loading In[946], in expression starting on line 8

In [ ]:
# build an animated gif
gif(anim, fps = 10)

In [940]:
# update the net representation with weights, etc
update!(viz)

In [ ]:
p1 = plot(rand(20))
p2 = plot(rand(10))
p3 = scatter(rand(100))
p4 = plot(rand(1000))

In [ ]:
subplot(p1,p2,p3,p4, nr=1, leg=false)

In [ ]:
immerse()
p = plot(rand(10))

In [ ]:
gui()

In [ ]:
append!(p,1,rand(10))
gui()